# DB연동
- DB연동하는 것은 에러가 없어야 함
- 해당 코드는 무조건 돌아가게끔 만들어야 함
- MySQL에 문제가 생김
  + 대처방안 : try-catch 구문

In [2]:
numerator = 10
denominator = 0

result = numerator / denominator

# 하고 싶은 것은 결과 확인 출력을 꼭 하고 싶음, 어떤 일이 있어도
print("결과 확인")

ZeroDivisionError: division by zero

In [4]:
try :
    numerator = 10
    denominator = 0
    result = numerator / denominator
except ZeroDivisionError:
    print("Error: 0으로 나눌 수 없음")
else:
    print("결과확인 : ", rusult)
finally:
    print("결과 확인")

Error: 0으로 나눌 수 없음
결과 확인


In [13]:
try :
    numerator = 10
    denominator = int(input("숫자를 입력하세요!!"))
    result = numerator / denominator
except ZeroDivisionError:
    print("Error: 0으로 나눌 수 없음")
except ValueError:
    print("invalid literal for int() with")
else:
    print("결과확인 : ", result)
finally:
    print("결과 확인")

숫자를 입력하세요!! adf


invalid literal for int() with
결과 확인


# Step 1. Connect ot Server

In [15]:
import os
from mysql import connector

PASSWORD = '1234'

- 기초문법 중급 레벨 ( with vs python decorator)

In [16]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD
    ) as database:
        print(f"Database 객체 : {database}")
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000016A3E052450>


# Step 2. Creating New DB

In [17]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD
    ) as database:
        print(f"Database 객체 : {database}")
        
        # 쿼리문 작성
        create_db = "CREATE DATABASE book_ratings;"
        with database.cursor() as cursor:
            cursor.execute(create_db)

            # 생성된 DB 확인 쿼리문 쓰기
            show_existing_db = "SHOW DATABASES;"
            cursor.execute(show_existing_db)
            for db in cursor:
                print(db)

except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000016A3E050350>
('book_ratings',)
('classicmodels',)
('information_schema',)
('instacart',)
('modeldb',)
('mydata',)
('mydaya2',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('testdb',)
('titanic',)
('world',)


- 특정 DB에 직접 연결하겠습니다!

In [19]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD,
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000016A3E051610>


# Step 3. Creating Tables

In [20]:
create_books_table = """
    CREATE TABLE books(
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        title VARCHAR(100),
        author VARCHAR(100),
        genre VARCHAR(100),
        release_year YEAR(4)
    );
"""

try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD,
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # cursor dbject 접근 후, 위 쿼리 실행
        with database.cursor() as cursor:
            cursor.execute(create_books_table)
            database.commit()

            # 테이블 확인 DESCRIBE
            describe_books = "DESCRIBE books;"
            cursor.execute(describe_books)
            book_schema = cursor.fetchall()
            for column in book_schema:
                print(column)
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000016A3E05B590>
('id', 'int', 'NO', 'PRI', None, 'auto_increment')
('title', 'varchar(100)', 'YES', '', None, '')
('author', 'varchar(100)', 'YES', '', None, '')
('genre', 'varchar(100)', 'YES', '', None, '')
('release_year', 'year', 'YES', '', None, '')


# Step 4. Data Insertion
- 하나의 레코드 추가할 때와 다중 레코드 추가할 때 사용하는 메서드 다름

In [ ]:
# INSERET INTO 테이블명 (컬럼명1, 컬럼명2, 컬럼명3....) VALUES (%s, %s, ...)

In [21]:
insert_single_record = "INSERT INTO books (id, title, author, genre, release_year)\
    VALUES (%s, %s, %s, %s, %s)"
single_record = (
    "1", "Choose Yourself! Be Happy, Make Millions, Live the Dream", "James Altucher", "self-help", "2013"
    )

In [22]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD,
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:
            cursor.execute(insert_single_record, single_record)
            database.commit()
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000016A3E017A90>


- 다중 레코드를 추가하자

In [23]:
insert_multiple_records = "INSERT INTO books (id, title, author, genre, release_year)\
    VALUES (%s, %s, %s, %s, %s)"
multiple_records = [
    (
        "2", 
        "Skip the Line: The 10,000 Experiments Rule and Other Surprising Advice for Reaching Your Goals",
        "James Altucher",
        "self-help",
        "2021"        
    ),
    (
        "3",
        "The Power of No: Because One Little Word Can Bring Health, Abundance, and Happiness",
        "James Altucher",
        "self-help",
        "2014"
    ),
    (
        "4",
        "The 48 Laws of Power",
        "Robert Greene",
        "self-help",
        "1998"
    ),
    (
        "5",
        "Mastery",
        "Robert Greene",
        "self-help",
        "2012"
    ),
    (
        "6",
        "The Art of Seduction",
        "Robert Greene",
        "self-help",
        "2001"
    ),
]

In [24]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD,
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:
            # cursor.execute(insert_single_record, single_record)
            cursor.executemany(insert_multiple_records, multiple_records)
            database.commit()
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000016A3E052650>


# Step 5. Selecting Data

In [33]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD,
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:
            # 쿼리문 작성
            select_cols = "SELECT author, release_year FROM books;"
            cursor.execute(select_cols)

            # 데이터 확인
            df = cursor.fetchall()
            columns = [desc[0] for desc in cursor.description]  # 컬럼 이름 가져오기
            df = pd.DataFrame(df, columns=columns)
            df
           
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000016A669A3D90>


## SELECT 사용자 정의 함수 만들기
- 어떤 테이블을 조회해도 깔끔하게 데이터프레임으로 만드는 함수를 만드시고,
- 테스트를 해주세요

In [ ]:
def tbtodf() :

In [26]:
import pandas as pd

In [34]:
df

,author,release_year
0,James Altucher,2013
1,James Altucher,2021
2,James Altucher,2014
3,Robert Greene,1998
4,Robert Greene,2012
5,Robert Greene,2001


In [40]:
def fetch_table_to_dataframe(database_name, table_name, host='localhost', user='root', password='1234'):
    try:
        with connector.connect(
            host=host,
            user=user,
            password=password,
            database=database_name
        ) as database:
            print(f"Database 객체 : {database}")

            # 커서 생성 후, 쿼리 실행
            with database.cursor() as cursor:
                # 쿼리문 작성
                query = f"SELECT * FROM {table_name};"
                cursor.execute(query)

                # 데이터 확인
                rows = cursor.fetchall()
                columns = [desc[0] for desc in cursor.description]  # 컬럼 이름 가져오기
                df = pd.DataFrame(rows, columns=columns)
                return df
               
    except connector.Error as e:
        print(e)
        return None

# 테스트 함수 호출
database_name = 'testdb'
table_name = 'tb_customer'

df = fetch_table_to_dataframe(database_name, table_name)
df

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000016A669EF610>


,CUSTOMER_CD,CUSTOMER_NM,MW_FLG,BIRTH_DAY,PHONE_NUMBER,EMAIL,TOTAL_POINT,REG_DTTM
0,2017042,강원진,M,19810603,002-8202-8790,wjgang@navi.com,280300,20170123132432
1,2017053,나경숙,W,19891225,002-4509-0043,ksna#boram.co.kr,4500,20170210180930
2,2017108,박승대,M,19711430,,sdpark@haso.com,23450,20170508203450
3,2018087,서유리,W,19810925,003-1265-8372,urseo@epnt.co.kr,18700,20180204160903
4,2018254,이혜옥,W,19839012,003_1287_9734,hylee@hansoft.com,570,20180619230805
5,2019001,김진숙,W,20010426,002-9842-0074,"jskim$dreami,org",12820,20190101080518
6,2019069,김한길,M,19920315,,hkkim@ssoya.com,15320,20190217110704
7,2019095,남궁소망,M,19620728,003-6273-8539,,890,20190312124558
8,2019167,한찬희,M,19711106,002=1202=5563,chhan@ecom.co.kr,6800,20190508155600
9,2019281,이아름,W,19940513,003-2620-0723,aulee@hoki.com,35600,20190709201308


# Step 6. Update and Delete
-

In [42]:
update_query = """
    UPDATE
        books
    SET
        author = 'Big J'
    WHERE
        author = 'James Altucher'
"""

try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD,
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:
            cursor.execute(update_query)
            database.commit()

            check_query = "SELECT DISTINCT author FROM books"
            cursor.execute(check_query)

            for result in cursor.fetchall():
                print(result)
                
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000016A3E05BF90>
('Big J',)
('Robert Greene',)


# Step 7. Delete Records

In [43]:
try:
    with connector.connect(
        host = 'localhost',
        user = 'root',
        password = PASSWORD,
        database = "book_ratings" # 이것이 기존 코드와의 차이점
    ) as database:
        print(f"Database 객체 : {database}")

        # 커서 생성 후, 쿼리 실행
        with database.cursor() as cursor:

            delete_query = "DELETE FROM books WHERE release_year <= 2010;"
            cursor.execute(delete_query)
            database.commit()

            check_query = "SELECT DISTINCT release_year FROM books"
            cursor.execute(check_query)
            for result in cursor.fetchall():
                print(result)
                
except connector.Error as e:
    print(e)

Database 객체 : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000016A67019790>
(2013,)
(2021,)
(2014,)
(2012,)
